In [1]:
!pip install pyngrok
!pip install wurlitzer
!pip uninstall keras -y
!pip install keras==2.15.0


Found existing installation: keras 3.1.1
Uninstalling keras-3.1.1:
  Successfully uninstalled keras-3.1.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.5 MB/s eta 0:00:0000:010:01


In [2]:
# Clone the github repository and navigate to the project directory.
!git clone https://github.com/AI4Bharat/IndicTrans2
%cd /kaggle/working/IndicTrans2/huggingface_interface
!python3 -m pip install nltk sacremoses pandas regex mock transformers>=4.33.2 mosestokenizer
!python3 -c "import nltk; nltk.download('punkt')"
!python3 -m pip install bitsandbytes scipy accelerate datasets
!python3 -m pip install sentencepiece
!git clone https://github.com/VarunGumma/IndicTransTokenizer
%cd IndicTransTokenizer
!python3 -m pip install --editable ./

Cloning into 'IndicTrans2'...
remote: Enumerating objects: 668, done.
remote: Counting objects: 100% (151/151), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 668 (delta 108), reused 77 (delta 70), pack-reused 517
Receiving objects: 100% (668/668), 4.12 MiB | 18.92 MiB/s, done.
Resolving deltas: 100% (415/415), done.
/kaggle/working/IndicTrans2/huggingface_interface
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 12.5 MB/s eta 0:00:0000:0100:01
Cloning into 'IndicTransTokenizer'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 123 (delta 52), reused 94 (delta 31), pack-reused 0
Receiving objects: 100% (123/123), 3.85 MiB | 20.03 MiB/s, done.
Resolving deltas: 100% (52/52), done.
/kaggle/working/IndicTrans2/huggingface_interface/IndicTra

In [3]:
import torch
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig
from IndicTransTokenizer import IndicProcessor, IndicTransTokenizer

BATCH_SIZE = 4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
quantization = None

In [4]:
def initialize_model_and_tokenizer(ckpt_dir, direction, quantization):
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    tokenizer = IndicTransTokenizer(direction=direction)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    if qconfig == None:
        model = model.to(DEVICE)
        if DEVICE == "cuda":
            model.half()

    model.eval()

    return tokenizer, model


def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip):
    translations = []
    for i in range(0, len(input_sentences), BATCH_SIZE):
        batch = input_sentences[i : i + BATCH_SIZE]

        # Preprocess the batch and extract entity mappings
        batch = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)

        # Tokenize the batch and generate input encodings
        inputs = tokenizer(
            batch,
            src=True,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(DEVICE)

        # Generate translations using the model
        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1,
            )

        # Decode the generated tokens into text
        generated_tokens = tokenizer.batch_decode(generated_tokens.detach().cpu().tolist(), src=False)

        # Postprocess the translations, including entity replacement
        translations += ip.postprocess_batch(generated_tokens, lang=tgt_lang)

        del inputs
        torch.cuda.empty_cache()

    return translations

In [5]:
en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-dist-200M"  # ai4bharat/indictrans2-en-indic-dist-200M
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir, "en-indic", quantization)

ip = IndicProcessor(inference=True)

en_sents = [
    "When I was young, I used to go to the park every day.",
    "He has many old books, which he inherited from his ancestors.",
]

src_lang, tgt_lang = "eng_Latn", "tam_Taml"
hi_translations = batch_translate(en_sents, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)

print(f"\n{src_lang} - {tgt_lang}")
for input_sentence, translation in zip(en_sents, hi_translations):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

del en_indic_tokenizer, en_indic_model

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

configuration_indictrans.py:   0%|          | 0.00/14.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-dist-200M:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_indictrans.py:   0%|          | 0.00/61.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-dist-200M:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]


eng_Latn - tam_Taml
eng_Latn: When I was young, I used to go to the park every day.
tam_Taml: நான் இளமையாக இருந்தபோது, ஒவ்வொரு நாளும் பூங்காவுக்குச் செல்வது வழக்கம்.
eng_Latn: He has many old books, which he inherited from his ancestors.
tam_Taml: அவர் தனது மூதாதையர்களிடமிருந்து மரபுரிமையாகப் பெற்ற பல பழைய புத்தகங்கள் அவரிடம் உள்ளன.


In [6]:
# Initialize the model and tokenizer
en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-dist-200M"  # Change to your actual checkpoint directory
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir, "en-indic", quantization)  # Change the quantization if needed
ip = IndicProcessor(inference=True)  # Initialize your processor here

In [ ]:
from flask import Flask, jsonify, request
from pyngrok import ngrok

app = Flask(__name__)
ngrok.set_auth_token("2eOCN9e1AgJi9oIB6MfQCfKZUpd_5rXsgn1oTq6gKoMXQ2j2g")

@app.route("/translate", methods=["POST"])
def translate():
    
    data = request.json
    title = data.get('title')
    body = data.get('body')
    target_lang = data.get('target_lang')
    
    src_lang = "eng_Latn"
    
    
    if not title or not body:
        return jsonify({"error": "Request must contain title and body fields."}), 400
    
    
    input_sentences = [title, body]

    # Translate
    translations = batch_translate(input_sentences, src_lang, target_lang, en_indic_model, en_indic_tokenizer, ip)

    
    translated_data = {
        'title': translations[0],
        'body': translations[1]
    }
    return jsonify(translated_data), 200

# ngrok tunnel
if __name__ == "__main__":
    public_url = ngrok.connect(5000)
    print(" * ngrok tunnel \"{}\" has been opened".format(public_url))
    
    # Run Flask app
    app.run(port=5000)
    
    
    ngrok.disconnect(public_url)


 * ngrok tunnel "NgrokTunnel: "https://7ffe-34-73-98-209.ngrok-free.app" -> "http://localhost:5000"" has been opened
 * Serving Flask app '__main__'
 * Debug mode: off


In [ ]:
# from flask import Flask, jsonify, request
# from pyngrok import ngrok

# app = Flask(__name__)


# port = 5000
# ngrok.set_auth_token("2eOCN9e1AgJi9oIB6MfQCfKZUpd_5rXsgn1oTq6gKoMXQ2j2g")
# @app.route("/")
# def hello():
#     return "Hello World!! from Google Colab"

# # Start ngrok when app is run
# if __name__ == "__main__":
#     # Setup ngrok
#     public_url = ngrok.connect(port)
#     print(" * ngrok tunnel \"{}\" has been opened".format(public_url))
    
#     # Provide the Flask app to run on the specified port
#     app.run(port=port)
    
#     ngrok.disconnect(public_url)
